# Gemini Pro 1.5 Test 

In [1]:
from vertexai.generative_models import HarmBlockThreshold
from vertexai.generative_models import GenerationConfig
from vertexai.generative_models import GenerativeModel
from vertexai.generative_models import HarmCategory
from vertexai.generative_models import Part
import vertexai 
import logging 
import os 

##### Setup logging 

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [3]:
logger.info(f'Using vertexai=={vertexai.__version__}')

Using vertexai==1.52.0


##### Setup essentials 

In [4]:
PROJECT_ID = 'arun-genai-bb'
LOCATION = 'us-central1'
MODEL_NAME = 'gemini-1.5-pro-preview-0514'

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './../../credentials/key.json'
vertexai.init(project=PROJECT_ID, location=LOCATION)

##### Setup model

In [6]:
with open('./../../data/prompt/system_instruction.txt', 'r') as file:
    lines = file.readlines()

system_instruction = [line.strip() for line in lines]
system_instruction

['You are a helpful language translator.',
 'Your mission is to translate text in English to French.']

In [7]:
model = GenerativeModel(MODEL_NAME, 
                        system_instruction=system_instruction)
model.__dict__

{'_model_name': 'publishers/google/models/gemini-1.5-pro-preview-0514',
 '_prediction_resource_name': 'projects/arun-genai-bb/locations/us-central1/publishers/google/models/gemini-1.5-pro-preview-0514',
 '_location': 'us-central1',
 '_generation_config': None,
 '_safety_settings': None,
 '_tools': None,
 '_tool_config': None,
 '_system_instruction': ['You are a helpful language translator.',
  'Your mission is to translate text in English to French.']}

##### Setup generation configs 

For deterministic output, we can set low values for top k, such as 1 or 2. We can also set a very low value for top p, close to 0. A low temperature near 0 will also ensure the responses are the most probable and deterministic.

In [8]:
generation_config = GenerationConfig(temperature=0.0, 
                                     top_p=0.0, 
                                     top_k=1, 
                                     candidate_count=1, 
                                     max_output_tokens=8192)
generation_config.__dict__

{'_raw_generation_config': temperature: 0
 top_p: 0
 top_k: 1
 candidate_count: 1
 max_output_tokens: 8192}

##### Set safety settings 

In [9]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}

##### Test

In [13]:
prompt = """User input: I like bagels."""
contents = [prompt]

In [14]:
model.count_tokens(contents)

total_tokens: 7
total_billable_characters: 22

In [15]:
response = model.generate_content(contents, 
                                  generation_config=generation_config, 
                                  safety_settings=safety_settings)
response

candidates {
  content {
    role: "model"
    parts {
      text: "J\'aime les bagels. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.158564851
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0802132487
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0824015141
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.10576342
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.133528158
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0829941481
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.228156507
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.138228953
  }
}
usage_metadata {
  prompt_token_count: 25
  candidates_token_count: 8
  t

##### Decipher the output response

In [17]:
response.text.strip()

"J'aime les bagels."

In [18]:
response.to_dict().get("usage_metadata")

{'prompt_token_count': 25,
 'candidates_token_count': 8,
 'total_token_count': 33}

In [19]:
response.candidates[0].finish_reason

<FinishReason.STOP: 1>

In [20]:
response.candidates[0].safety_ratings

[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.158564851
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0802132487
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0824015141
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.10576342
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.133528158
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0829941481
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.228156507
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.138228953
]